# model

> A PyTorch implementation of the [YOLOX](https://arxiv.org/abs/2107.08430) object detection model based on [OpenMMLab](https://github.com/open-mmlab)’s implementation in the [mmdetection](https://github.com/open-mmlab/mmdetection) library.

In [ ]:
#| default_exp model

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
from typing import Any, Type, List, Optional, Callable, Tuple
from functools import partial

from pathlib import Path

In [ ]:
#| export
import torch
import torch.nn as nn

import torch.nn.init as init

In [ ]:
#| export
from cjm_yolox_pytorch.utils import multi_apply

In [ ]:
#| export
MODEL_TYPES = ['yolox_tiny', 'yolox_s', 'yolox_m', 'yolox_l', 'yolox_x']

CSP_DARKNET_CFGS = {
    MODEL_TYPES[0]:dict(deepen_factor=0.33, widen_factor=0.375),
    MODEL_TYPES[1]:dict(deepen_factor=0.33, widen_factor=0.5),
    MODEL_TYPES[2]:dict(deepen_factor=0.67, widen_factor=0.75),
    MODEL_TYPES[3]:dict(deepen_factor=1.0, widen_factor=1.0),
    MODEL_TYPES[4]:dict(deepen_factor=1.33, widen_factor=1.25)
}

PAFPN_CFGS = {
    MODEL_TYPES[0]:dict(in_channels=[96, 192, 384], out_channels=96, num_csp_blocks=1),
    MODEL_TYPES[1]:dict(in_channels=[128, 256, 512], out_channels=128, num_csp_blocks=1),
    MODEL_TYPES[2]:dict(in_channels=[192, 384, 768], out_channels=192, num_csp_blocks=2),
    MODEL_TYPES[3]:dict(in_channels=[256, 512, 1024], out_channels=256, num_csp_blocks=3),
    MODEL_TYPES[4]:dict(in_channels=[320, 640, 1280], out_channels=320, num_csp_blocks=4),
}

HEAD_CFGS = {
    MODEL_TYPES[0]:dict(in_channels=96,feat_channels=96),
    MODEL_TYPES[1]:dict(in_channels=128,feat_channels=128),
    MODEL_TYPES[2]:dict(in_channels=192, feat_channels=192),
    MODEL_TYPES[3]:dict(in_channels=256, feat_channels=256),
    MODEL_TYPES[4]:dict(in_channels=320, feat_channels=320),
}

OPENMMLAB_CKPT_URL = 'https://download.openmmlab.com/mmdetection/v2.0/yolox'

PRETRAINED_URLS = {
    MODEL_TYPES[0]:f'{OPENMMLAB_CKPT_URL}/yolox_tiny_8x8_300e_coco/yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth',
    MODEL_TYPES[1]:f'{OPENMMLAB_CKPT_URL}/yolox_s_8x8_300e_coco/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth',
    MODEL_TYPES[2]:None,
    MODEL_TYPES[3]:f'{OPENMMLAB_CKPT_URL}/yolox_l_8x8_300e_coco/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth',
    MODEL_TYPES[4]:f'{OPENMMLAB_CKPT_URL}/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth',
}

NORM_CFG = dict(momentum=0.03, eps=0.001)

MODEL_CFGS = {model_type: {**CSP_DARKNET_CFGS[model_type], 
                            **{'neck_'+k: v for k, v in PAFPN_CFGS[model_type].items()}, 
                            **{'head_'+k: v for k, v in HEAD_CFGS[model_type].items()}, 
                            **{k:{"pretrained": v != None} for k,v in PRETRAINED_URLS.items()}[model_type]} 
               for model_type in MODEL_TYPES}

In [ ]:
model_type = MODEL_TYPES[0]
model_type

'yolox_tiny'

In [ ]:
#| export
class ConvModule(nn.Module):
    """
    Configurable block used for Convolution2d-Normalization-Activation blocks.
    
    #### Pseudocode
    Function forward(input x):
    
        1. Pass the input (x) through the convolutional layer and store the result back to x.
        2. Pass the output from the convolutional layer (now stored in x) through the batch normalization layer and store the result back to x.
        3. Apply the activation function to the output of the batch normalization layer (x) and return the result.

    """

    def __init__(self, 
                 in_channels: int,  # Number of channels in the input image
                 out_channels: int, # Number of channels produced by the convolution
                 kernel_size: int,  # Size of the convolving kernel
                 stride: int = 1,   # Stride of the convolution.
                 padding: int = 0,  # Zero-padding added to both sides of the input.
                 bias: bool = True, # If set to False, the layer will not learn an additive bias.
                 eps: float = 1e-05,    # A value added to the denominator for numerical stability in BatchNorm2d.
                 momentum: float = 0.1, # The value used for the running_mean and running_var computation in BatchNorm2d.
                 affine: bool = True,   # If set to True, this module has learnable affine parameters.
                 track_running_stats: bool = True, # If set to True, this module tracks the running mean and variance.
                 activation_function: Type[nn.Module] = nn.SiLU # The activation function to be applied after batch normalization.
                ):
        
        super(ConvModule, self).__init__()

        # Convolutional layer
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=bias)
        # Batch normalization layer
        self.bn = nn.BatchNorm2d(out_channels, eps=eps, momentum=momentum, affine=affine, track_running_stats=track_running_stats)
        # Activation function
        self.activate = activation_function()
        
        init.kaiming_normal_(self.conv.weight.data, mode='fan_out', nonlinearity='relu')

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # Pass input through convolutional layer
        x = self.conv(x)
        # Pass output from convolutional layer through batch normalization
        x = self.bn(x)
        # Apply activation function and return result
        return self.activate(x)

In [ ]:
#| export
class DarknetBottleneck(nn.Module):
    """
    Basic Darknet bottleneck block used in Darknet.
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/models/utils/csp_layer.py#L8)
    
    #### Pseudocode
    Function forward(input_tensor x):
    
    1. Store x as identity.
    2. Pass x through the first convolutional layer (conv1), and then the result through the second convolutional layer (conv2). Store the output as 'out'.
    3. Check if add_identity is True:
       a. If True, check if identity_conv exists (i.e., when the input and output channels do not match).
          i. If identity_conv exists, pass identity through the identity_conv layer and update the identity.
       b. Add the updated identity to the 'out'.
    4. Return 'out' as the final output.

    """
    
    def __init__(self, 
                 in_channels: int, # The number of input channels to the block.
                 out_channels: int, # The number of output channels from the block.
                 eps: float = 0.001, # A value added to the denominator for numerical stability in the ConvModule's BatchNorm layer.
                 momentum: float = 0.03, # The value used for the running_mean and running_var computation in the ConvModule's BatchNorm layer.
                 affine: bool = True, # A flag that when set to True, gives the ConvModule's BatchNorm layer learnable affine parameters.
                 track_running_stats: bool = True, # If True, the ConvModule's BatchNorm layer will track the running mean and variance.
                 add_identity: bool = True # If True, add an identity shortcut (also known as skip connection) to the output.
                ):
        super(DarknetBottleneck, self).__init__()

        self.add_identity = add_identity
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # First convolutional layer with 1x1 kernel size
        self.conv1 = ConvModule(in_channels, out_channels, kernel_size=1, stride=1, padding=0, 
                                bias=False, eps=eps, momentum=momentum, affine=affine, 
                                track_running_stats=track_running_stats)
        
        # Second convolutional layer with 3x3 kernel size
        self.conv2 = ConvModule(out_channels, out_channels, kernel_size=3, stride=1, padding=1, 
                                bias=False, eps=eps, momentum=momentum, affine=affine, 
                                track_running_stats=track_running_stats)
        
        # Add a 1x1 conv on shortcut when in and out channels don't match
        if self.add_identity and self.in_channels != self.out_channels:
            self.identity_conv = ConvModule(in_channels, out_channels, kernel_size=1, stride=1, padding=0, 
                                            bias=False, eps=eps, momentum=momentum, affine=affine, 
                                            track_running_stats=track_running_stats)
        else:
            self.identity_conv = None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
                
        # Store the input tensor as identity tensor for possible use in shortcut connection
        identity = x
        # Pass the input tensor through two conv layers in sequence
        out = self.conv2(self.conv1(x))
        
        # If add_identity is True, then add the original input (identity) to the output of the conv layers
        if self.add_identity:
            # Apply a conv layer to the identity tensor if the in and out channels don't match
            if self.identity_conv is not None:
                identity = self.identity_conv(identity)
            # Add the identity tensor to the output tensor
            out += identity

        return out

In [ ]:
#| export
class CSPLayer(nn.Module):
    
    """
    Cross Stage Partial Layer
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/models/utils/csp_layer.py#L75)
    
    #### Pseudocode
    1. Start with input `x`
    2. Apply `main_conv` layer on `x` and store the output in `x1`
    3. Apply `short_conv` layer on `x` and store the output in `x2`
    4. For each `block` in the `blocks` list:
        - Pass `x1` through the `block` and store the result back in `x1`
    5. Concatenate `x1` and `x2` along dimension 1 to form a new tensor called `out`
    6. Pass `out` through the `final_conv` layer and store the result back in `out`
    7. Return `out` as the final output of the forward function
    
    Notes:
    
    - The `main_conv`, `short_conv`, and `final_conv` are convolution layers which apply a series of transformations (including convolution, batch normalization, and non-linear activation) on the input tensor.
    - The `blocks` list contains a number of "bottleneck" layers that perform additional transformations on the `x1` tensor. Each bottleneck layer has a specific structure, with two convolution layers and an optional identity shortcut.
    - The concatenation operation combines the results of the `main_conv` layers processed through `blocks` and the `short_conv` layer, preparing the tensor for the final transformations in `final_conv`.
    """
        
    def __init__(self, 
                 in_channels, # Number of input channels.
                 out_channels, # Number of output channels.
                 num_blocks, # Number of blocks in the bottleneck.
                 kernel_size=1, # Size of the convolving kernel.
                 stride=1, # Stride of the convolution.
                 padding=0, # Zero-padding added to both sides of the input.
                 eps=0.001, # A value added to the denominator for numerical stability in the ConvModule's BatchNorm layer.
                 momentum=0.03, # The value used for the running_mean and running_var computation in the ConvModule's BatchNorm layer.
                 affine=True, # A flag that when set to True, gives the ConvModule's BatchNorm layer learnable affine parameters.
                 track_running_stats=True, # Whether or not to track the running mean and variance during training.
                 add_identity=True # Whether or not to add an identity shortcut connection if the input and output are the same size.
                ):
        
        super(CSPLayer, self).__init__()

        # Define the number of hidden_channels as half the number of out_channels
        hidden_channels = out_channels // 2

        # Define the main convolution layer
        self.main_conv = ConvModule(in_channels, hidden_channels, kernel_size, stride, padding, 
                                    bias=False, eps=eps, momentum=momentum, affine=affine, 
                                    track_running_stats=track_running_stats)

        # Define the short convolution layer
        self.short_conv = ConvModule(in_channels, hidden_channels, kernel_size, stride, padding, 
                                     bias=False, eps=eps, momentum=momentum, affine=affine, 
                                     track_running_stats=track_running_stats)

        # Define the final convolution layer, which takes in the concatenated output from main_conv 
        # and short_conv and outputs to the final output layer
        self.final_conv = ConvModule(2 * hidden_channels, out_channels, kernel_size, stride, padding, 
                                     bias=False, eps=eps, momentum=momentum, affine=affine, 
                                     track_running_stats=track_running_stats)
        
        # Define a list of blocks using the DarknetBottleneck structure
        self.blocks = nn.ModuleList([DarknetBottleneck(hidden_channels, hidden_channels, eps, momentum, affine, track_running_stats, add_identity) for _ in range(num_blocks)])

    def forward(self, x):
        
        # Pass the input through the main_conv layer
        x1 = self.main_conv(x)

        # Pass the input through the short_conv layer
        x2 = self.short_conv(x)

        # Pass the output of main_conv through each block in the blocks list
        for block in self.blocks:
            x1 = block(x1)

        # Concatenate the outputs of the blocks and the short_conv along dimension 1
        out = torch.cat((x1, x2), dim=1)

        # Pass the concatenated output through the final_conv layer
        out = self.final_conv(out)

        # Returning the final output
        return out

In [ ]:
#| export
class Focus(nn.Module):
    
    """
    Focus width and height information into channel space.
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/models/backbones/csp_darknet.py#L14)
    
    
    #### Pseudocode
    Function forward(input_tensor x):

    1. Split the input tensor x into four patches (patch_top_left, patch_top_right, patch_bot_left, patch_bot_right) based on their spatial positions in the last two dimensions of x.
    2. Concatenate these four patches along the channel dimension and store the output back into 'x'.
    3. Pass 'x' through the convolutional layer (self.conv) and store the result as 'out'.
    4. Return 'out' as the final output.

    """
    
    def __init__(self,
                 in_channels: int, # Number of input channels.
                 out_channels: int, # Number of output channels.
                 kernel_size: int = 1, # Size of the convolving kernel.
                 stride: int = 1, # Stride of the convolution.
                 bias: bool = False, # If set to False, the layer will not learn an additive bias.
                 eps: float = 0.001, #  A value added to the denominator for numerical stability in the ConvModule's BatchNorm layer.
                 momentum: float = 0.03, # The value used for the running_mean and running_var computation in the ConvModule's BatchNorm layer.
                 affine: bool = True, # A flag that when set to True, gives the ConvModule's BatchNorm layer learnable affine parameters.
                 track_running_stats: bool = True # Whether or not to track the running mean and variance during training.
                ):
        
        super(Focus, self).__init__()
        self.conv = ConvModule(
            in_channels * 4,
            out_channels,
            kernel_size,
            stride,
            padding=(kernel_size - 1) // 2,
            bias=bias,
            eps=eps,
            momentum=momentum,
            affine=affine,
            track_running_stats=track_running_stats)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
                
        # Split the input tensor into 4 patches
        patch_top_left = x[..., ::2, ::2]   # Top left patch
        patch_top_right = x[..., ::2, 1::2]  # Top right patch
        patch_bot_left = x[..., 1::2, ::2]  # Bottom left patch
        patch_bot_right = x[..., 1::2, 1::2]  # Bottom right patch
        
        # Concatenate the patches along the channel dimension
        x = torch.cat(
            (
                patch_top_left,
                patch_bot_left,
                patch_top_right,
                patch_bot_right,
            ),
            dim=1,
        )
        return self.conv(x)

In [ ]:
#| export
class SPPBottleneck(nn.Module):
    """
    Spatial Pyramid Pooling layer used in YOLOv3-SPP
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/models/backbones/csp_darknet.py#L67)
    
    #### Pseudocode
    Function forward(input_tensor x):

    1. Pass x through the first convolutional layer (conv1) and assign the output back to x.
    2. Create an empty list called pooling_results and add x to it.
    3. For each pooling layer in the poolings list, do the following:
       a. Apply the pooling layer on x and append the output to the pooling_results list.
    4. Concatenate all tensors in the pooling_results list along the channel dimension (dimension 1) and assign the output back to x.
    5. Pass x through the second convolutional layer (conv2) to combine the features from all pooling layers and project them to the desired number of output channels.
    6. Return the final output tensor.

    """

    def __init__(self, 
                 in_channels: int, # The number of input channels.
                 out_channels: int, # The number of output channels.
                 pool_sizes: List[int]=[5, 9, 13], # The sizes of the pooling areas.
                 eps: float=0.001, # A value added to the denominator for numerical stability in the BatchNorm layer.
                 momentum: float=0.03, #  The value used for the running_mean and running_var computation in the BatchNorm layer.
                 affine: bool=True, # A flag that when set to True, gives the BatchNorm layer learnable affine parameters.
                 track_running_stats: bool=True # Whether to keep track of running mean and variance in BatchNorm.
                ):
        
        super(SPPBottleneck, self).__init__()
        
        # Reducing the number of channels by a factor of 2
        hidden_channels = in_channels // 2

        # Convolution layer to reduce the number of channels
        self.conv1 = ConvModule(in_channels, hidden_channels, kernel_size=1, stride=1, padding=0, 
                                bias=False, eps=eps, momentum=momentum, affine=affine, 
                                track_running_stats=track_running_stats)

        # List of pooling layers with different window sizes
        self.poolings = nn.ModuleList([nn.MaxPool2d(kernel_size=ps, stride=1, padding=ps//2) for ps in pool_sizes])

        # Convolution layer to combine the features from the pooling layers and project them to the desired number of output channels
        self.conv2 = ConvModule(hidden_channels*(len(pool_sizes)+1), out_channels, kernel_size=1, stride=1, padding=0, 
                                bias=False, eps=eps, momentum=momentum, affine=affine, 
                                track_running_stats=track_running_stats)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
                
        # Reducing the number of channels by a factor of 2 using the convolution layer
        x = self.conv1(x)

        # Applying max pooling with different window sizes and collecting the results
        pooling_results = [x]
        for pooling in self.poolings:
            pooling_results.append(pooling(x))

        # Concatenating the results of pooling along the channel dimension
        x = torch.cat(pooling_results, dim=1)

        # Combining the features and projecting them to the desired number of output channels using the convolution layer
        return self.conv2(x)

In [ ]:
#| export
class CSPDarknet(nn.Module):
    """
    CSP-Darknet backbone
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/models/backbones/csp_darknet.py#L124)
    
    #### Pseudocode
    Function forward(input_tensor x):

    1. Initialize an empty list 'outs' to store intermediate outputs.
    2. For each index and layer name in the model's layers:
       a. Retrieve the layer corresponding to the current layer name.
       b. Pass 'x' through the retrieved layer and update 'x' with the output.
       c. If the current index is in the set of output indices:
          i. Append 'x' to 'outs'.
    3. Convert 'outs' to a tuple and return it as the final output.

    """

    # Architecture settings for P5 and P6
    ARCH_SETTINGS = {
        'P5': [[64, 128, 3, True, False], [128, 256, 9, True, False],
               [256, 512, 9, True, False], [512, 1024, 3, False, True]],
        'P6': [[64, 128, 3, True, False], [128, 256, 9, True, False],
               [256, 512, 9, True, False], [512, 768, 3, True, False],
               [768, 1024, 3, False, True]]
    }

    def __init__(self,
                 arch='P5', # Architecture configuration, 'P5' or 'P6'.
                 deepen_factor=1.0, # Factor to adjust the number of channels in each layer.
                 widen_factor=1.0, # Factor to adjust the number of blocks in CSP layer.
                 out_indices=(2, 3, 4), # Indices of the stages to output.
                 spp_kernal_sizes=(5, 9, 13), # Sizes of the pooling operations in the Spatial Pyramid Pooling.
                 momentum=0.03, # Momentum for the moving average in batch normalization.
                 eps=0.001 # Epsilon for batch normalization to avoid numerical instability.
                ):
        
        super().__init__()

        if not set(out_indices).issubset(range(len(self.ARCH_SETTINGS[arch]) + 1)):
            raise ValueError("out_indices are out of range")

        self.out_indices = out_indices
        # Building the initial layer of the model
        self.stem = Focus(
            3,
            int(self.ARCH_SETTINGS[arch][0][0] * widen_factor),
            kernel_size=3,
            stride=1
            )
        self.layers = ['stem']
        # Building the stages of the model
        self._build_stages(arch, deepen_factor, widen_factor, spp_kernal_sizes, momentum, eps)

    def _build_stages(self, arch, deepen_factor, widen_factor, spp_kernal_sizes, momentum, eps):
        """
        Build the stages of the CSPDarknet model.

        Args:
            arch (str): Architecture type, 'P5' or 'P6'.
            deepen_factor (float): Factor to adjust the depth of the model.
            widen_factor (float): Factor to adjust the width of the model.
            spp_kernal_sizes (tuple): Sizes of the pooling operations in the Spatial Pyramid Pooling.
            momentum (float): Momentum for the moving average in batch normalization.
            eps (float): Epsilon for batch normalization to avoid numerical instability.
        """

        # For each stage configuration in the architecture settings
        for i, (in_c, out_c, num_blocks, add_identity, use_spp) in enumerate(self.ARCH_SETTINGS[arch]):
            # Adjust the channel size based on the widen factor
            in_c, out_c = int(in_c * widen_factor), int(out_c * widen_factor)
            # Adjust the number of blocks based on the deepen factor
            num_blocks = max(round(num_blocks * deepen_factor), 1)

            stage = []

            # Append ConvModule for the stage
            stage.append(ConvModule(in_c, 
                                    out_c, 3, 
                                    stride=2, 
                                    padding=1, 
                                    bias=False, 
                                    eps=eps, 
                                    momentum=momentum, 
                                    affine=True, 
                                    track_running_stats=True))

            # If use_spp is True, append a Spatial Pyramid Pooling layer
            if use_spp:
                stage.append(SPPBottleneck(out_c, out_c, pool_sizes=spp_kernal_sizes))

            # Append a Cross Stage Partial layer
            stage.append(CSPLayer(out_c, out_c, num_blocks=num_blocks, add_identity=add_identity))
            # Add the stage to the model as a sequential layer
            self.add_module(f'stage{i + 1}', nn.Sequential(*stage))
            self.layers.append(f'stage{i + 1}')

    def forward(self, x):
                
        outs = []
        # For each layer in the model
        for i, layer_name in enumerate(self.layers):
            # Get the layer by its name
            layer = getattr(self, layer_name)
            # Pass the input through the layer
            x = layer(x)
            # If the index is in out_indices, append the output to outs
            if i in self.out_indices:
                outs.append(x)
        return tuple(outs)

In [ ]:
csp_darknet_cfg = CSP_DARKNET_CFGS[model_type]
csp_darknet = CSPDarknet(**csp_darknet_cfg)

backbone_inp = torch.randn(1, 3, 256, 256)

with torch.no_grad():
    backbone_out = csp_darknet(backbone_inp)
[out.shape for out in backbone_out]

[torch.Size([1, 96, 32, 32]),
 torch.Size([1, 192, 16, 16]),
 torch.Size([1, 384, 8, 8])]

In [ ]:
#| export
class YOLOXPAFPN(nn.Module):
    """
    Path Aggregation Feature Pyramid Network (PAFPN) used in YOLOX.
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/models/necks/yolox_pafpn.py#L14)
    
    
    #### Pseudocode
    Function forward(list inputs):

    1. Assert that the length of inputs equals the length of in_channels.
    2. Initialize inner_outs with the last feature map in inputs.
    3. For each index in in_channels in reverse (exclude first):
       a. Set feature_high to the first feature map in inner_outs.
       b. Set feature_low to the corresponding input feature map.
       c. Reduce the channels of feature_high using the corresponding reduce_layer. Update feature_high in inner_outs.
       d. Upsample feature_high to the spatial size of feature_low.
       e. Concatenate upsampled feature_high and feature_low, and pass the result through the corresponding CSPLayer. The output is inner_out.
       f. Add inner_out to the beginning of inner_outs.
    4. Initialize outs with the first feature map from inner_outs.
    5. For each index in in_channels (exclude last):
       a. Set feature_low to the last feature map in outs.
       b. Set feature_high to the corresponding feature map in inner_outs.
       c. Downsample feature_low to the spatial size of feature_high.
       d. Concatenate downsampled feature_low and feature_high, and pass the result through the corresponding CSPLayer. The output is out.
       e. Add out to the end of outs.
    6. For each convolutional layer (conv) in out_convs:
       a. Apply conv to the corresponding feature map in outs.
    7. Return the tuple of feature maps in outs as the final output.
    """
    def __init__(self,
                 in_channels, # The number of input channels for each level of the feature pyramid.
                 out_channels, # The number of output channels for each level of the feature pyramid.
                 num_csp_blocks=3, # The number of bottlenecks in each CSPLayer.
                 upsample_cfg=dict(scale_factor=2, mode='nearest'), # The configuration for the upsampling operation.
                 momentum=0.03, # The momentum for the batch normalization in the ConvModule.
                 eps=0.001 # The epsilon for the batch normalization in the ConvModule.
                ):
        super(YOLOXPAFPN, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        # build top-down blocks, which includes reduce layers and CSP blocks
        self.upsample = nn.Upsample(**upsample_cfg)
        self.reduce_layers = nn.ModuleList()
        self.top_down_blocks = nn.ModuleList()
        for idx in range(len(in_channels) - 1, 0, -1):
            # add reducing layers for channel-wise feature reduction
            self.reduce_layers.append(
                ConvModule(
                    in_channels[idx],
                    in_channels[idx - 1],
                    kernel_size=(1, 1), 
                    stride=(1, 1),
                    padding=0,
                    bias=False,
                    momentum=momentum,
                    eps=eps,
                    affine=True, 
                    track_running_stats=True
                ))
            # add CSP layers for feature learning
            self.top_down_blocks.append(
                CSPLayer(
                    in_channels[idx - 1] * 2,
                    in_channels[idx - 1],
                    num_blocks=num_csp_blocks,
                    add_identity=False))

        # build bottom-up blocks, which includes downsampling layers and CSP blocks
        self.downsamples = nn.ModuleList()
        self.bottom_up_blocks = nn.ModuleList()
        for idx in range(len(in_channels) - 1):
            # add downsampling layers for spatial reduction
            self.downsamples.append(
                ConvModule(
                    in_channels[idx],
                    in_channels[idx],
                    3,
                    stride=2,
                    padding=1,
                    bias=False,
                    momentum=momentum,
                    eps=eps,
                    affine=True, 
                    track_running_stats=True
                ))
            # add CSP layers for feature learning
            self.bottom_up_blocks.append(
                CSPLayer(
                    in_channels[idx] * 2,
                    in_channels[idx + 1],
                    num_blocks=num_csp_blocks,
                    add_identity=False))

        # build output convolutions for each level
        self.out_convs = nn.ModuleList()
        for i in range(len(in_channels)):
            self.out_convs.append(
                ConvModule(
                    in_channels[i],
                    out_channels,
                    1,
                    stride=(1, 1),
                    padding=0,
                    bias=False,
                    momentum=momentum,
                    eps=eps,
                    affine=True, 
                    track_running_stats=True))

    def forward(self, inputs):
        
        assert len(inputs) == len(self.in_channels)

        # top-down path
        inner_outs = [inputs[-1]]
        for idx in range(len(self.in_channels) - 1, 0, -1):
            feat_heigh = inner_outs[0]
            feat_low = inputs[idx - 1]
            # reduce the channel dimension of the higher feature map
            feat_heigh = self.reduce_layers[len(self.in_channels) - 1 - idx](feat_heigh)
            inner_outs[0] = feat_heigh

            # upsample the higher feature map to the same spatial size as the lower one
            upsample_feat = self.upsample(feat_heigh)

            # concatenate the upsampled higher feature map and the lower one, and feed them into the CSPLayer
            inner_out = self.top_down_blocks[len(self.in_channels) - 1 - idx](
                torch.cat([upsample_feat, feat_low], 1))
            # collect the output feature maps
            inner_outs.insert(0, inner_out)

        # bottom-up path
        outs = [inner_outs[0]]
        for idx in range(len(self.in_channels) - 1):
            feat_low = outs[-1]
            feat_height = inner_outs[idx + 1]
            # downsample the lower feature map to the same spatial size as the higher one
            downsample_feat = self.downsamples[idx](feat_low)
            # concatenate the downsampled lower feature map and the higher one, and feed them into the CSPLayer
            out = self.bottom_up_blocks[idx](
                torch.cat([downsample_feat, feat_height], 1))
            # collect the output feature maps
            outs.append(out)

        # apply the output convolutions to the feature maps
        for idx, conv in enumerate(self.out_convs):
            outs[idx] = conv(outs[idx])

        return tuple(outs)

In [ ]:
pafpn_cfg = PAFPN_CFGS[model_type]
yolox_pafpn = YOLOXPAFPN(**pafpn_cfg)

with torch.no_grad():
    neck_out = yolox_pafpn(backbone_out)
[out.shape for out in neck_out]

[torch.Size([1, 96, 32, 32]),
 torch.Size([1, 96, 16, 16]),
 torch.Size([1, 96, 8, 8])]

In [ ]:
#| export
class YOLOXHead(nn.Module):
    """
    The head of YOLOX model <https://arxiv.org/abs/2107.08430>, used for bounding box prediction.
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/models/dense_heads/yolox_head.py#L20)
    
    #### Pseudocode
    Function forward(feats):

    1. For each scale level in feats, perform the following steps:
        a. Pass the scale level feature through the classification convolutions (cls_convs) to produce a feature map (cls_feat).
        b. Pass the same scale level feature through the regression convolutions (reg_convs) to produce another feature map (reg_feat).
        c. Apply the classification predictor convolution (conv_cls) on the classification feature map (cls_feat) to get the classification scores (cls_score).
        d. Apply the regression predictor convolution (conv_reg) on the regression feature map (reg_feat) to get bounding box predictions (bbox_pred).
        e. Apply the objectness predictor convolution (conv_obj) on the regression feature map (reg_feat) to get objectness scores (objectness).
    2. Collect the classification scores, bounding box predictions, and objectness scores for each scale level and store them in a tuple.
    3. Return the tuple as the final output.

    """

    BBOX_DIM = 4
    OBJECTNESS_DIM = 1

    def __init__(self,
                 num_classes:int, # The number of target classes.
                 in_channels:int, # The number of input channels.
                 feat_channels=256, # The number of feature channels.
                 stacked_convs=2, # The number of convolution layers to stack.
                 strides=[8, 16, 32], # The stride of each scale level in the feature pyramid.
                 momentum=0.03, # The momentum for the moving average in batch normalization.
                 eps=0.001 # The epsilon to avoid division by zero in batch normalization.
                ):

        super().__init__()
        self.num_classes = num_classes
        self.cls_out_channels = num_classes
        self.in_channels = in_channels
        self.feat_channels = feat_channels
        self.stacked_convs = stacked_convs
        self.strides = strides
        self.momentum = momentum
        self.eps = eps
        self._init_layers()  # initialize layers

    def _init_layers(self):
        """
        Initialize layers for the head module, includes classification 
        convolutions, regression convolutions and prediction convolutions 
        for each scale level.
        """
        self.multi_level_cls_convs = nn.ModuleList()
        self.multi_level_reg_convs = nn.ModuleList()
        self.multi_level_conv_cls = nn.ModuleList()
        self.multi_level_conv_reg = nn.ModuleList()
        self.multi_level_conv_obj = nn.ModuleList()
        for _ in self.strides:
            self.multi_level_cls_convs.append(self._build_stacked_convs())
            self.multi_level_reg_convs.append(self._build_stacked_convs())
            conv_cls, conv_reg, conv_obj = self._build_predictor()
            self.multi_level_conv_cls.append(conv_cls)
            self.multi_level_conv_reg.append(conv_reg)
            self.multi_level_conv_obj.append(conv_obj)

    def _build_stacked_convs(self):
        """
        Build stacked convolution layers.

        Returns
        -------
        nn.Sequential
            A sequential container of stacked conv layers.
        """
        conv = ConvModule
        stacked_convs = []
        for i in range(self.stacked_convs):
            chn = self.in_channels if i == 0 else self.feat_channels
            stacked_convs.append(
                conv(
                    chn,
                    self.feat_channels,
                    3,
                    stride=1,
                    padding=1,
                    momentum=self.momentum,
                    eps=self.eps,
                    affine=True, 
                    track_running_stats=True,
                    bias=False))
        return nn.Sequential(*stacked_convs)

    def _build_predictor(self):
        """
        Build predictor layers for classification, regression, and objectness.

        Returns
        -------
        tuple
            The classification, regression, and objectness convolutional layers.
        """
        conv_cls = nn.Conv2d(self.feat_channels, self.cls_out_channels, 1)
        conv_reg = nn.Conv2d(self.feat_channels, self.BBOX_DIM, 1)
        conv_obj = nn.Conv2d(self.feat_channels, self.OBJECTNESS_DIM, 1)
        return conv_cls, conv_reg, conv_obj

    def forward_single(self, x, cls_convs, reg_convs, conv_cls, conv_reg,
                       conv_obj):
        """
        Forward feature of a single scale level.

        Parameters
        ----------
        x : tensor
            The input tensor.
        cls_convs : nn.Module
            The classification convolutions.
        reg_convs : nn.Module
            The regression convolutions.
        conv_cls : nn.Module
            The classification predictor convolution.
        conv_reg : nn.Module
            The regression predictor convolution.
        conv_obj : nn.Module
            The objectness predictor convolution.

        Returns
        -------
        tuple
            The classification scores, bounding box predictions, and objectness.
        """
        cls_feat = cls_convs(x)
        reg_feat = reg_convs(x)

        cls_score = conv_cls(cls_feat)
        bbox_pred = conv_reg(reg_feat)
        objectness = conv_obj(reg_feat)

        return cls_score, bbox_pred, objectness

    def forward(self, feats):
        """
        Forward pass for the head.

        Parameters
        ----------
        feats : list
            A list of multi-level features.

        Returns
        -------
        tuple
            The results of applying the forward_single function to each scale level.
        """
        return multi_apply(self.forward_single, feats,
                           self.multi_level_cls_convs,
                           self.multi_level_reg_convs,
                           self.multi_level_conv_cls,
                           self.multi_level_conv_reg,
                           self.multi_level_conv_obj)


In [ ]:
head_cfg = HEAD_CFGS[model_type]
yolox_head = YOLOXHead(num_classes=80, **head_cfg)

with torch.no_grad():
    cls_scores, bbox_preds, objectness = yolox_head(neck_out)    
print(f"cls_scores: {[cls_score.shape for cls_score in cls_scores]}")
print(f"bbox_preds: {[bbox_pred.shape for bbox_pred in bbox_preds]}")
print(f"objectness: {[objectness.shape for objectness in objectness]}")

cls_scores: [torch.Size([1, 80, 32, 32]), torch.Size([1, 80, 16, 16]), torch.Size([1, 80, 8, 8])]
bbox_preds: [torch.Size([1, 4, 32, 32]), torch.Size([1, 4, 16, 16]), torch.Size([1, 4, 8, 8])]
objectness: [torch.Size([1, 1, 32, 32]), torch.Size([1, 1, 16, 16]), torch.Size([1, 1, 8, 8])]


In [ ]:
#| export
class YOLOX(nn.Module):
    """
    Implementation of `YOLOX: Exceeding YOLO Series in 2021`
    
    - <https://arxiv.org/abs/2107.08430>
    
    #### Pseudocode
    Function forward(input_tensor x):

    1. Pass x through the backbone module. The backbone module performs feature extraction from the input images. Store the output as 'x'.
    2. Pass the updated x through the neck module. The neck module performs feature aggregation of the extracted features. Update 'x' with the new output.
    3. Pass the updated x through the bbox_head module. The bbox_head module predicts bounding boxes for potential objects in the images using the aggregated features. Update 'x' with the new output.
    4. Return 'x' as the final output. The final 'x' represents the model's predictions for object locations within the input images.

    """
    def __init__(self, 
                 backbone:CSPDarknet, # Backbone module for feature extraction.
                 neck:YOLOXPAFPN, # Neck module for feature aggregation.
                 bbox_head:YOLOXHead): # Bbox head module for predicting bounding boxes.
        super(YOLOX, self).__init__()

        self.backbone = backbone
        self.neck = neck
        self.bbox_head = bbox_head

    def forward(self, x):
        # Forward through backbone
        x = self.backbone(x)

        # Forward through neck
        x = self.neck(x)

        # Forward through bbox_head
        x = self.bbox_head(x)

        return x

In [ ]:
yolox = YOLOX(csp_darknet, yolox_pafpn, yolox_head)

with torch.no_grad():
    cls_scores, bbox_preds, objectness = yolox(backbone_inp)    
print(f"cls_scores: {[cls_score.shape for cls_score in cls_scores]}")
print(f"bbox_preds: {[bbox_pred.shape for bbox_pred in bbox_preds]}")
print(f"objectness: {[objectness.shape for objectness in objectness]}")

cls_scores: [torch.Size([1, 80, 32, 32]), torch.Size([1, 80, 16, 16]), torch.Size([1, 80, 8, 8])]
bbox_preds: [torch.Size([1, 4, 32, 32]), torch.Size([1, 4, 16, 16]), torch.Size([1, 4, 8, 8])]
objectness: [torch.Size([1, 1, 32, 32]), torch.Size([1, 1, 16, 16]), torch.Size([1, 1, 8, 8])]


In [ ]:
#| export
def kaiming_init(module:torch.nn.Module # The module to be initialized.
                ) -> None:
    """
    Initializes the weights of the Conv2d layers of the given model using the [Kaiming Normal initialization](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_normal_).
    """
    
    # If the module is a 2d convolutional layer
    if isinstance(module, nn.Conv2d):
        
        # Apply Kaiming Normal initialization to the weights of the module
        # We use 'fan_out' mode as this preserves the magnitude of the variance of the weights
        # in the forward pass. The nonlinearity is set to 'relu' as the network uses ReLU activation functions.
        init.kaiming_normal_(module.weight.data, mode='fan_out', nonlinearity='relu')

In [ ]:
list(yolox_head.parameters())[0][0][0]

tensor([[ 0.0403,  0.0276, -0.0600],
        [ 0.0094,  0.0068,  0.0484],
        [-0.0257,  0.0385, -0.0589]], grad_fn=<SelectBackward0>)

In [ ]:
kaiming_init(yolox_head)
list(yolox_head.parameters())[0][0][0]

tensor([[ 0.0403,  0.0276, -0.0600],
        [ 0.0094,  0.0068,  0.0484],
        [-0.0257,  0.0385, -0.0589]], grad_fn=<SelectBackward0>)

In [ ]:
#| export
def init_head(head: YOLOXHead, # The YOLOX head to be initialized.
              num_classes: int # The number of classes in the dataset.
             ) -> None:
    """
    Initialize the `YOLOXHead` with appropriate class outputs and convolution layers.

    This function configures the output channels in the YOLOX head to match the
    number of classes in the dataset. It also initializes multiple level
    convolutional layers for each stride in the YOLOX head.
    """

    # Set the number of output channels in the head to be equal to the number of classes
    head.cls_out_channels = num_classes

    # Create a list of 2D convolutional layers, one for each stride in the head.
    # Each convolutional layer will have a number of output channels equal to the number of classes
    # and a kernel size of 1 (i.e., it will perform a 1x1 convolution).
    
    conv_layers = [nn.Conv2d(head.feat_channels, head.cls_out_channels, 1) for _ in head.strides]
    
    for conv in conv_layers:
        init.kaiming_normal_(conv.weight.data, mode='fan_out', nonlinearity='relu')
    
    head.multi_level_conv_cls = nn.ModuleList(conv_layers)
    
    # Use Kaiming initialization to initialize the weights of the convolutional layers. 
    head.multi_level_conv_cls.apply(kaiming_init)

In [ ]:
yolox_head.multi_level_conv_cls

ModuleList(
  (0-2): 3 x Conv2d(96, 80, kernel_size=(1, 1), stride=(1, 1))
)

In [ ]:
init_head(yolox_head, 19)
yolox_head.multi_level_conv_cls

ModuleList(
  (0-2): 3 x Conv2d(96, 19, kernel_size=(1, 1), stride=(1, 1))
)

In [ ]:
#| export
from cjm_psl_utils.core import download_file

In [ ]:
#| export
def build_model(model_type:str, # Type of the model to be built.
                num_classes:int, # Number of classes for the model.
                pretrained:bool=True, # Whether to load pretrained weights.
                checkpoint_dir:str='./pretrained_checkpoints/' # Directory to store checkpoints.
               ) -> YOLOX: # The built YOLOX model.
    """
    Builds a YOLOX model based on the given parameters.
    """
    
    assert model_type in MODEL_TYPES, f"Invalid model_type. Expected one of: {MODEL_TYPES}, but got {model_type}"

    backbone_cfg = CSP_DARKNET_CFGS[model_type]
    neck_cfg = PAFPN_CFGS[model_type]
    head_cfg = HEAD_CFGS[model_type]
    
    backbone = CSPDarknet(**backbone_cfg)
    neck = YOLOXPAFPN(**neck_cfg)

    if pretrained and PRETRAINED_URLS[model_type] == None:
        print("The selected model type does not have a pretrained checkpoint. Initializing model with untrained weights.")
        pretrained = False
    
    try:
        if pretrained:
            url = PRETRAINED_URLS[model_type]
            checkpoint_path = os.path.join(checkpoint_dir, Path(url).name)
            download_file(url, checkpoint_dir)
            
            pretrained_ckpt = torch.load(checkpoint_path)['state_dict']
            num_pretrained_classes = pretrained_ckpt['bbox_head.multi_level_conv_cls.0.weight'].shape[0]
            
            head = YOLOXHead(num_classes=num_pretrained_classes, **head_cfg)
        else:
            head = YOLOXHead(num_classes=num_classes, **head_cfg)
        
        yolox = YOLOX(backbone, neck, head)
        
        if pretrained:
            yolox.load_state_dict(pretrained_ckpt)
            init_head(head, num_classes)
            
    except Exception as e:
        print(f"Error occurred while building the model: {str(e)}")
        return None

    return yolox

In [ ]:
yolox = build_model(model_type, 19, pretrained=True)

test_inp = torch.randn(1, 3, 256, 256)

with torch.no_grad():
    cls_scores, bbox_preds, objectness = yolox(test_inp)
    
print(f"cls_scores: {[cls_score.shape for cls_score in cls_scores]}")
print(f"bbox_preds: {[bbox_pred.shape for bbox_pred in bbox_preds]}")
print(f"objectness: {[objectness.shape for objectness in objectness]}")

The file ./pretrained_checkpoints/yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth already exists and overwrite is set to False.
cls_scores: [torch.Size([1, 19, 32, 32]), torch.Size([1, 19, 16, 16]), torch.Size([1, 19, 8, 8])]
bbox_preds: [torch.Size([1, 4, 32, 32]), torch.Size([1, 4, 16, 16]), torch.Size([1, 4, 8, 8])]
objectness: [torch.Size([1, 1, 32, 32]), torch.Size([1, 1, 16, 16]), torch.Size([1, 1, 8, 8])]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()